<h1><center>Homemade implementations of Extension 1 - LDA with Ledoit-Wolf</center></h1>

### Imports

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Get the wine data

In [2]:
wine = load_wine()
X_all = wine.data
y_all = wine.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=0.8, random_state=0)

### Get the iris data

In [4]:
data = load_iris()
petal_length = data.data[:,2].reshape(-1,1)
petal_width = data.data[:,3].reshape(-1,1)
X_all_iris = np.hstack((petal_length, petal_width))  # only using 2 attributes
y_all_iris = data.target

In [5]:
X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(X_all_iris, y_all_iris, train_size=0.8, random_state=0)

### Homemade Ledoit-Wolf LDA implementation

In [6]:
def get_class_priors(y): # return dict in which keys are classes and values are proportions of each class in the data
    priors = dict()
    for c in set(y):
        priors[c] = np.sum(y == c) / y.shape[0]
    return priors

In [7]:
def ledoit_wolf(X):
    n, d = X.shape
    cov = np.cov(X, rowvar=False, bias=1)                         # the paper calls this Sn, defined at the beginning of section 3.2 on page 377
    mn = np.trace(cov) / d                                        # lemma 3.2 on page 379, the brackets are defined on page 368
    dn = np.sum(np.square(cov - mn))                              # lemma 3.3 on page 379
    b_bar = (1 / (n ** 2)) * np.sum(np.square(X.T @ X / n - cov)) # top of page 380
    bn = min(b_bar, dn)                                           # top of page 380
    return (bn / dn) * mn + ((dn - bn) / dn) * cov                # lemma 3.5 on page 380

In [8]:
def get_lda_covar(X, y, priors, ledoit=False):
    if not ledoit: # plain LDA uses the weighted sum of the covariance matrices of each class, aka the scatter matrix
        covs = [priors[c] * np.cov(X[y==c], rowvar=False, bias=1) for c in priors]
        return np.add.reduce(covs)
    else:
        return ledoit_wolf(X)

In [9]:
def get_lda_means(X, y):
    answer = dict()
    for c in set(y):
        answer[c] = np.mean(X[y == c], axis=0)
    return answer

In [10]:
def lda_pdf(x, mu, cov): # the usual multivariate Gaussian pdf 
    scale = 1 / (((2 * np.pi)**((mu.shape[0])/2)) * np.sqrt(np.linalg.det(cov)))
    part2 = (-1/2) * ((x-mu).T @ np.linalg.inv(cov) @ (x-mu))
    return scale * np.exp(part2)

In [11]:
def lda_classify(X_train, y_train, X_test, ledoit):
    priors = get_class_priors(y_train)
    mus = get_lda_means(X_train, y_train)
    cov = get_lda_covar(X_train, y_train, priors, ledoit)
    predictions = []
    
    for example in X_test:
        best_class = None
        best_likelihood = -1
        for c in priors:
            curr_likelihood = priors[c] * lda_pdf(example, mus[c], cov)
            if curr_likelihood > best_likelihood:
                best_likelihood = curr_likelihood
                best_class = c
        predictions.append(best_class)
    
    return predictions

# Evaluate my homemade implementation's accuracy

### i) Check the accuracy of my Ledoit-Wolf LDA on the wine data

In [12]:
ledoit_wolf_pred = lda_classify(X_train, y_train, X_test, True)
print(f"The accuracy of my Ledoit-Wolf LDA on the wine data is {100 * accuracy_score(y_test, ledoit_wolf_pred)}")

The accuracy of my Ledoit-Wolf LDA on the wine data is 100.0


### ii) Check the accuracy of my Ledoit-Wolf LDA on the iris data

In [13]:
ledoit_wolf_pred_iris = lda_classify(X_train_iris, y_train_iris, X_test_iris, True)
print(f"The accuracy of Ledoit-Wolf LDA on the iris dataset is {100 * accuracy_score(y_test_iris, ledoit_wolf_pred_iris)}")

The accuracy of Ledoit-Wolf LDA on the iris dataset is 93.33333333333333
